In [ ]:
import torch
import faiss
import numpy as np
import pdfplumber
import os
import cv2
# Import tensorflow with CPU support if GPU is not available or causing issues
# import tensorflow as tf  # Original import
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'  # Force TensorFlow to use CPU
import tensorflow as tf
from tqdm import tqdm
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import google.generativeai as genai
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model


# Configure Gemini API
genai.configure(api_key="key")  # Replace with your Gemini API key

# Load the trained image classification model
model_path = r"/content/dish_classification_model1.h5"  # Ensure the correct path
model = load_model(model_path)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Define class labels
class_labels = ['Biryani', 'Butter chicken', 'Chapati', 'Chicken tandoori', 'Chole bhature',  
 'Dal makhani', 'Dhokla', 'Dosa', 'Gajar halwa', 'Ghevar',  
 'Gulab jamun', 'Jalebi', 'Kadai paneer', 'Kathi roll', 'Kebabs',  
 'Kofta', 'Masala bhindi', 'Medu vada', 'Pani puri', 'Pav bhaji',  
 'Poori', 'Rasgulla', 'Samosa', 'Toor dal', 'Vada pav']

# Function to preprocess the image
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (128, 128))
    image = img_to_array(image) / 255.0
    image = np.expand_dims(image, axis=0)
    return image

# Function to make predictions
def predict_image(image_path):
    image = preprocess_image(image_path)
    predictions = model.predict(image)
    class_index = np.argmax(predictions)
    confidence = np.max(predictions)
    predicted_class = class_labels[class_index]
    return predicted_class, confidence

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
    return text.strip()

# Function to process PDFs and find relevant dish
def process_pdfs(pdf_folder, dish_name):
    pdf_files = [f for f in os.listdir(pdf_folder) if f.endswith(".pdf")]
    matching_pdf = None
    for pdf_file in pdf_files:
        if dish_name.lower() in pdf_file.lower():
            matching_pdf = pdf_file
            break
    
    if not matching_pdf:
        return None, None

    text = extract_text_from_pdf(os.path.join(pdf_folder, matching_pdf))
    return matching_pdf, text

# Function to split text into chunks
def chunk_text(text, chunk_size=2048, chunk_overlap=400):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    return text_splitter.split_text(text)

# Function to create FAISS vector store
def create_vector_store(pdf_name, text):
    embedding_model = SentenceTransformer("BAAI/bge-base-en-v1.5")
    dimension = 768
    index = faiss.IndexFlatL2(dimension)

    chunks = chunk_text(text)
    embeddings = [embedding_model.encode(chunk, convert_to_numpy=True, normalize_embeddings=True) for chunk in chunks]
    
    if embeddings:
        index.add(np.array(embeddings, dtype=np.float32))
    
    return index, embedding_model, chunks

# Function to retrieve relevant chunks
def retrieve_relevant_chunks(query, index, embedding_model, chunks, top_k=3):
    query_embedding = embedding_model.encode(query, convert_to_numpy=True, normalize_embeddings=True).astype(np.float32)
    _, indices = index.search(np.array([query_embedding]), top_k)
    return [chunks[i] for i in indices[0] if i < len(chunks)]

# Function to generate response using Gemini API
def generate_response(prompt):
    system_prompt = (
        "You are a professional chef providing complete and well-structured recipes. "
        "Ensure the response includes all ingredients and steps in a clear, logical order."
    )
    final_prompt = f"{system_prompt}\n\n{prompt}"
    model = genai.GenerativeModel("gemini-1.5-pro-latest")
    response = model.generate_content(final_prompt)
    return response.text.strip()

# Main function to integrate image classification and RAG
def main(image_path):
    pdf_folder = "./recipes_pdf"
    
    if not os.path.exists(pdf_folder):
        print(f"Folder '{pdf_folder}' does not exist. Please upload PDFs.")
        return
    
    # Step 1: Predict the dish name from the image
    predicted_dish, confidence = predict_image(image_path)
    print(f"Predicted Dish: {predicted_dish} (Confidence: {confidence:.2f})")
    
    # Step 2: Retrieve the recipe using RAG
    pdf_name, text = process_pdfs(pdf_folder, predicted_dish)
    
    if not text:
        print(f"No relevant content found for '{predicted_dish}'.")
        return
    
    index, embedding_model, chunks = create_vector_store(pdf_name, text)
    retrieved_texts = retrieve_relevant_chunks(f"Recipe for {predicted_dish}", index, embedding_model, chunks)
    
    if not retrieved_texts:
        print(f"No relevant content found for '{predicted_dish}'.")
        return
    
    context = "\n".join(retrieved_texts)
    print(f"\nRetrieved content from {pdf_name}:")
    print(context[:500] + "...")  # Display only the first 500 characters
    
    prompt = f"Recipe for {predicted_dish}:\n\n{context}\n\nProvide a well-structured step-by-step guide."
    response = generate_response(prompt)
    
    print(f"\nGenerated Recipe for {predicted_dish}:")
    print(response)

if __name__ == "__main__":
    image_path = r"/content/dal_makhani.jpeg"  # Update with your image path
    main(image_path)